In [62]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_keys

#Configure gmaps
gmaps.configure(api_key=g_keys)

In [20]:
vacation_df= pd.read_csv("weather_json.csv", index_col=0)
vacation_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudness,WindSpeed,Country,Date
0,kambove,-10.8764,26.5969,75.07,35,8,7.02,CD,1657879051
1,nanortalik,60.1432,-45.2371,43.68,83,100,0.65,GL,1657879051
2,kodiak,57.7900,-152.4072,54.66,91,90,3.98,US,1657879051
3,cherskiy,68.7500,161.3000,49.89,54,99,7.23,RU,1657879051
4,mataura,-46.1927,168.8643,38.48,98,98,2.86,NZ,1657879051


In [6]:
# Locations
locations = vacation_df[["Lat", "Lng"]]

humidity = vacation_df["Humidity"]

In [7]:
locations

,Lat,Lng
0,-10.8764,26.5969
1,60.1432,-45.2371
2,57.7900,-152.4072
3,68.7500,161.3000
4,-46.1927,168.8643
...,...,...
575,41.5703,120.4586
576,-9.2000,25.8500
577,62.7161,129.6658
578,-31.6333,-60.7000


In [9]:
humidity

0      35
1      83
2      91
3      54
4      98
       ..
575    57
576    26
577    59
578    91
579    38
Name: Humidity, Length: 580, dtype: int64

In [10]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [77]:
# Filtering DataFrame to find your ideal weather condition

ideal_weather_df= vacation_df.loc[(vacation_df["Max_Temp"] < 80) & (vacation_df["Max_Temp"] > 70) & (vacation_df["WindSpeed"] < 10) & (vacation_df["Cloudness"] == 0)]

In [78]:
ideal_weather_df.reset_index(drop=True)

,City,Lat,Lng,Max_Temp,Humidity,Cloudness,WindSpeed,Country,Date
0,sacramento,38.4666,-121.3177,70.93,66,0,1.99,US,1657878997
1,midland,31.9974,-102.0779,73.90,59,0,4.61,US,1657879063
2,laguna,38.4210,-121.4238,71.01,67,0,1.99,US,1657879078
3,tyler,32.3513,-95.3011,74.89,92,0,3.42,US,1657879081
4,belyy yar,53.6039,91.3903,73.29,33,0,6.71,RU,1657879088
5,menongue,-14.6585,17.6910,73.51,20,0,9.35,AO,1657879090
6,davidson,36.1501,-86.7833,71.69,90,0,0.00,US,1657879091
7,khani,41.9563,42.9566,71.17,33,0,4.94,GE,1657879094
8,kamariotissa,40.4833,25.4667,77.38,61,0,3.98,GR,1657879101


In [79]:
# Hotel variable
hotels = []

# Loop through filtered dataframe to get nearby hotel
for city in range(len(ideal_weather_df["City"])):

    lat = ideal_weather_df.loc[city]["Lat"]
    lng = ideal_weather_df.loc[city]["Lng"]

    target_coordinates = f"{lat}, {lng}"
    target_radius = 5000
    target_type = "lodging"

    params = {
        "location": target_coordinates, 
        "radius": target_radius,
        "types": target_type,
        "key": g_keys
         }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    response = requests.get(base_url, params=params)  

KeyError: 0